<a href="https://colab.research.google.com/github/arpanbiswas52/paper-code-microscopyGDM/blob/main/src/GDM_AFM(Notebook).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A Fourier-Based Global Denoising Model for Smart Artifacts Removing of Microscopy Images**
Oct, 2025

- Model developed by **Huanhuan Zhao** and **Arpan Biswas**
- Analysis by **Huanhuan Zhao** and **Arpan Biswas**
- Pantoea sp. YR343 biofilm data prepared by **Ruben Millan-Solsona** and **Spenser R. Brown**
- Project conceived and supervised by **Arpan Biswas** and **Wonhee Ko**


The notebook provides the workflow of our Global Denoising Model where we need 1-2 non-pair training images, generated from simulation or experiments or both.

- The integral architecture of the model builds on a lighweight Unet model

- The training images are prepared based on goal-specific pre-processing of raw simulated and experimental data, as per availability. The goal here is to dissolve the known artifacts and/or enhance the known features.

- The architecture has the flexibility to tune the weighting parameter which defines the percentage of information used from each image channel during training process

- Here, a FFT based loss function is also integrated with tradtional pixel based MSE loss function. Minimizing the FFT loss help to minimize the random noise and maximize the preservation of the key features.

- Here, we demonsrated the application on AFM-generated Pantoea sp. YR343 bio-film images


In [2]:
!pip install -q pillow tqdm matplotlib
!pip install brisque

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 7.3 MB/s eta 0:00:00
  Created wheel for libsvm-official: filename=libsvm_official-3.36.0-cp312-cp312-linux_x86_64.whl size=124635 sha256=16df04290359c529b18fda6c520442bf624e9e8c90e58b2dcdca52f570deb283
  Stored in directory: /root/.cache/pip/wheels/df/65/4b/c3cdece6e5fa7eebef116be2d5a309f7ac50c90183cbe12c92
Successfully built libsvm-official


In [3]:
import os, random
from glob import glob
from PIL import Image
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from brisque import BRISQUE


from skimage import io, img_as_float
#from skimage.metrics import niqe
import glob

from math import sqrt
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray

In [4]:
from google.colab import drive
import os
import shutil


# 1. Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# Some helper function to measure image quality without reference/ground truth image

In [5]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from skimage.feature import peak_local_max

def stm_fft_quality(image_input, peak_radius=10, bg_radius=30, patch_size=1, plot= False):
    # Load image in grayscale
    # Convert to grayscale NumPy array
    peak_radius = peak_radius/patch_size
    bg_radius = bg_radius/patch_size

    img_np = np.array(image_input)
    if img_np.ndim == 3:
        img = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    else:
        img = img_np
    img = img.astype(np.float32)

    # Normalize to [0,1]
    img /= img.max()

    # Compute FFT
    fft_img = np.fft.fftshift(np.fft.fft2(img))
    magnitude_spectrum = np.abs(fft_img)

    # Compute center (DC component)
    cy, cx = magnitude_spectrum.shape[0]//2, magnitude_spectrum.shape[1]//2

    # Create mask for peaks and background
    y, x = np.ogrid[:magnitude_spectrum.shape[0], :magnitude_spectrum.shape[1]]
    r = np.sqrt((x-cx)**2 + (y-cy)**2)

    # Signal region: frequencies between peak_radius and bg_radius
    signal_mask = (r >= peak_radius) & (r <= bg_radius)
    background_mask = (r > bg_radius)

    # Compute mean intensity in signal vs background
    signal_power = magnitude_spectrum[signal_mask].mean()
    noise_power = magnitude_spectrum[background_mask].mean()

    # FFT quality score = Signal-to-Noise Ratio (SNR) in dB
    snr_db = 10 * np.log10(signal_power / noise_power)

    if plot == True:
      # Visualization
      plt.figure(figsize=(10,4))
      plt.subplot(1,2,1)
      plt.imshow(img, cmap='gray')
      plt.title('STM Image')
      plt.axis('off')

      plt.subplot(1,2,2)
      plt.imshow(np.log1p(magnitude_spectrum), cmap='inferno')
      plt.title('FFT Magnitude Spectrum')
      plt.axis('off')
      plt.show()

    return snr_db


######################################################################################


# Upgraded version
def stm_fft_peak_quality(image_input, min_distance=10, peak_prominence=0.05, patch_size=1, win= True, plot= False):
    # Load STM image in grayscale
    # Convert to grayscale NumPy array
    peak_prominence = peak_prominence/patch_size
    min_distance = min_distance/patch_size

    img_np = np.array(image_input)
    if img_np.ndim == 3:
        img = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    else:
        img = img_np
    img = img.astype(np.float32)

    # Normalize to [0,1]
    img /= img.max()

    # Apply window to reduce edge artifacts
    if win == True:
      window = np.outer(np.hanning(img.shape[0]), np.hanning(img.shape[1]))
      img_win = img * window
    else:
      img_win = img

    # FFT and magnitude spectrum
    fft_img = np.fft.fftshift(np.fft.fft2(img_win))
    magnitude_spectrum = np.abs(fft_img)
    log_spectrum = np.log1p(magnitude_spectrum)

    # Detect peaks (exclude center DC peak)
    cy, cx = img.shape[0] // 2, img.shape[1] // 2
    coordinates = peak_local_max(
        log_spectrum,
        min_distance=int(min_distance),
        threshold_abs=int(peak_prominence * log_spectrum.max())
    )

    # Remove central DC component
    coordinates = [c for c in coordinates if np.hypot(c[0] - cy, c[1] - cx) > min_distance]

    # Compute peak power and noise floor
    peak_values = [magnitude_spectrum[y, x] for y, x in coordinates]
    if len(peak_values) == 0:
        print("No lattice peaks detected — image likely low quality.")
        return None

    peak_power = np.mean(peak_values)

    # Noise region: outside 50% of the max radius
    y, x = np.ogrid[:img.shape[0], :img.shape[1]]
    r = np.sqrt((x - cx)**2 + (y - cy)**2)
    noise_mask = r > 0.5 * r.max()
    noise_power = magnitude_spectrum[noise_mask].mean()

    # Peak-to-Noise Ratio (PNR) in dB
    pnr_db = 10 * np.log10(peak_power / noise_power)

    if plot == True:
      #Plot results
      plt.figure(figsize=(12,5))
      plt.subplot(1,2,1)
      plt.imshow(img, cmap='gray')
      plt.title("STM Image")
      plt.axis("off")

      plt.subplot(1,2,2)
      plt.imshow(log_spectrum, cmap='inferno')
      #plt.scatter([c[1] for c in coordinates], [c[0] for c in coordinates],
      #            marker='o', facecolors='none', edgecolors='cyan', s=80, label='Detected Peaks')
      plt.title("FFT Magnitude Spectrum + Peaks")
      plt.axis("off")
      plt.legend()
      plt.show()

    return pnr_db



In [6]:
# This function measure the quality of the images via FFT peak intensity plots and the calculated Peak-to-Noise (PNR) scores (higher the better)
def showFFTScore(img,den):
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))
    axes[0].imshow(img.convert("L"), cmap="gray")
    axes[0].set_title("Original")
    axes[0].axis("off")

    axes[1].imshow(den, cmap="gray")
    axes[1].set_title("Denoised")
    axes[1].axis("off")

    plt.suptitle("Test Image")
    plt.tight_layout()
    plt.show()

    # Check quality -BRISQUE # lower = better
    #obj = BRISQUE(url=False)
    #print("Original image (BRISQUE):" +str(obj.score(img)))
    #print("Denoise image (BRISQUE):" +str(obj.score(den.convert("RGB"))))

    score1 = stm_fft_quality(np.array(img), plot=True)
    print(f"Basic FFT-based quality score (SNR in dB): Original image: {score1:.2f} dB") #Higher is better
    score1 = stm_fft_quality(np.array(den), plot=True)
    print(f"Basic FFT-based quality score (SNR in dB): Denoise image: {score1:.2f} dB") #Higher is better

    score2 = stm_fft_peak_quality(np.array(img), plot=True)
    if score2 is not None:
      print(f"Upgraded FFT-based quality score (SNR in dB): Original image: {score2:.2f} dB") #Higher is better
    score2 = stm_fft_peak_quality(np.array(den), plot=True)
    if score2 is not None:
      print(f"Upgraded FFT-based quality score (SNR in dB): Denoise image: {score2:.2f} dB") #Higher is better


##################################################################################################################
# This function measure the quality of the images via detected lines plots
# and the calculated total line length within the range of angle, representing the artifacts (lower the better).
from skimage import feature
from skimage.transform import probabilistic_hough_line
import numpy as np
import matplotlib.pyplot as plt

def show_detected_lines_angle_range(img,
                                    sigma=0.1,
                                    low_threshold=1,
                                    high_threshold=10,
                                    line_length=1,
                                    line_gap=2,
                                    angle_min=-50,
                                    angle_max=-35):
    """
    Detect fine lines in STM-like images, draw only lines in the specified angle range,
    return the total length of lines within that range, and show histogram of all line angles.
    """

    # if not isinstance(img, np.ndarray):
    #     img = img.detach().cpu().numpy()
    img = np.array(img)
    img = img.astype(np.float32)
    if img.max() > 1:
        img = img / 255.0

    # Edge detection
    edges = feature.canny(img, sigma=sigma,
                          low_threshold=low_threshold/255.0,
                          high_threshold=high_threshold/255.0)

    # Hough line transform
    lines = probabilistic_hough_line(edges,
                                     threshold=5,
                                     line_length=line_length,
                                     line_gap=line_gap)

    total_length_in_range = 0.0
    angles = []
    lines_in_range = []
    lengths = []

    # Filter lines by angle
    for (p0, p1) in lines:
        dx = p1[0] - p0[0]
        dy = p1[1] - p0[1]
        angle = np.degrees(np.arctan2(dy, dx))
        length = np.sqrt(dx**2 + dy**2)
        angles.append(angle)
        lengths.append(length)
        if angle_min <= angle <= angle_max:
            total_length_in_range += length
            lines_in_range.append((p0, p1))

    # Plot results
    fig, axes = plt.subplots(2, 2, figsize=(18, 18))
    ax = axes.ravel()

    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Image")
    ax[0].axis("off")

    ax[1].imshow(edges, cmap="gray")
    for (p0, p1) in lines_in_range:
        ax[1].plot((p0[0], p1[0]), (p0[1], p1[1]), 'r-', linewidth=1)
    ax[1].set_title(f"Lines in [{angle_min}, {angle_max}]°\nTotal length = {total_length_in_range:.2f} px")
    ax[1].axis("off")

    ax[2].hist(angles, bins=36, range=(-90, 90), color="blue", alpha=0.7)
    ax[2].set_title("Histogram of Line Angles",fontsize=20)
    ax[2].set_xlabel("Angle (degrees)",fontsize=20)
    ax[2].set_ylabel("Count",fontsize=20)

    ax[3].hist(lengths, bins=np.arange(0, 101, 5), color="green", alpha=0.7)

    ax[3].set_title("Histogram of Line Lengths",fontsize=20)
    ax[3].set_xlabel("Line Length (pixels)",fontsize=20)
    ax[3].set_ylabel("Count",fontsize=20)
    plt.xticks(fontsize=20, fontfamily='monospace', fontstyle='italic')
    plt.yticks(fontsize=20, fontfamily='monospace', fontstyle='italic')


    plt.tight_layout()
    plt.show()

    return total_length_in_range


# FFT based loss functions

- In this paper, we have used the 2nd function

In [7]:
# Cal fft loss for training denoise model
def fft_loss(pred, target):
    # Apply FFT
    pred_fft = torch.fft.fft2(pred)
    target_fft = torch.fft.fft2(target)

    # Shift zero frequency to center
    pred_fft = torch.fft.fftshift(pred_fft)
    target_fft = torch.fft.fftshift(target_fft)

    # Use magnitude (ignore phase)
    pred_mag = torch.abs(pred_fft)
    target_mag = torch.abs(target_fft)

    # Normalize for stability
    pred_mag = pred_mag / (pred_mag.mean() + 1e-8)
    target_mag = target_mag / (target_mag.mean() + 1e-8)

    # L1 loss in frequency domain
    return torch.mean(torch.abs(pred_mag - target_mag))

# FFT-based quality score (higher is better, normalized cross-correlation in frequency domain)
def fft_quality(pred, target):
    """
    FFT-based cosine similarity between predicted and target images.
    Returns a score in [0, 1] (higher = better).
    """
    # Compute FFT
    pred_fft = torch.fft.fft2(pred)
    target_fft = torch.fft.fft2(target)

    # Shift zero-frequency to center (for interpretability)
    pred_fft = torch.fft.fftshift(pred_fft)
    target_fft = torch.fft.fftshift(target_fft)

    # Magnitude spectra
    pred_mag = torch.abs(pred_fft).flatten()
    target_mag = torch.abs(target_fft).flatten()

    # Cosine similarity in frequency domain
    score = torch.dot(pred_mag, target_mag) / (
        pred_mag.norm() * target_mag.norm() + 1e-8
    )

    return score.item()

# Denoising model architecture and training module

In [8]:
# -------------------- Simple U-Net model --------------------
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
    def forward(self, x): return self.net(x)

class UNetSmall(nn.Module):
    def __init__(self, in_ch=1, base=32):
        super().__init__()
        self.enc1 = DoubleConv(in_ch, base)
        self.enc2 = DoubleConv(base, base*2)
        self.enc3 = DoubleConv(base*2, base*4)
        self.pool = nn.MaxPool2d(2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)
        self.dec3 = DoubleConv(base*6, base*2)
        self.dec2 = DoubleConv(base*3, base)
        self.final = nn.Conv2d(base, in_ch, kernel_size=1)
    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        d3 = self.up(e3)
        d3 = torch.cat([d3, e2], dim=1)
        d3 = self.dec3(d3)
        d2 = self.up(d3)
        d2 = torch.cat([d2, e1], dim=1)
        d2 = self.dec2(d2)
        out = self.final(d2)
        return out

# Function to generate image patches from single training image
class STMNoiseSingleImageDataset(Dataset): #input is .png image
    def __init__(self, image_path, patch_size=128):
        if not os.path.exists(image_path):
            raise RuntimeError(f"File not found: {image_path}")
        self.img = Image.open(image_path).convert("L")
        self.patch_size = patch_size
        self.len = 1000  # number of random patches per epoch

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        w, h = self.img.size
        if w < self.patch_size or h < self.patch_size:
            self.img = self.img.resize((max(w, self.patch_size), max(h, self.patch_size)))
            w, h = self.img.size
        left = random.randint(0, w - self.patch_size)
        top = random.randint(0, h - self.patch_size)
        patch = self.img.crop((left, top, left + self.patch_size, top + self.patch_size))
        patch = np.array(patch).astype(np.float32) / 255.0
        patch = torch.from_numpy(patch).unsqueeze(0)
        return patch

# Function to create random mask on the image patches
def create_random_mask(batch, mask_fraction=0.005):
    B, C, H, W = batch.shape
    Npix = H * W
    nm = max(1, int(mask_fraction * Npix))
    masks = torch.zeros_like(batch)
    for b in range(B):
        idxs = torch.randperm(Npix)[:nm]
        ys = idxs // W
        xs = idxs % W
        masks[b, 0, ys, xs] = 1.0
    return masks

# Function to train the GDM model
def train(w1=0.5, w2=0.5, isfft=True):

    dataset1 = STMNoiseSingleImageDataset(train_image_path1, patch_size=patch_size)
    loader1 = DataLoader(dataset1, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

    dataset2 = STMNoiseSingleImageDataset(train_image_path2, patch_size=patch_size)  ### FIXED
    loader2 = DataLoader(dataset2, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

    model = UNetSmall(in_ch=1, base=32).to(device)
    optim = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=10, gamma=0.5)
    mse = nn.MSELoss(reduction='none')

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        running_fft_score = 0.0

        # zip ensures parallel iteration (stops at shortest loader)
        pbar = tqdm(zip(loader1, loader2), total=min(len(loader1), len(loader2)), desc=f"Epoch {epoch+1}/{epochs}")
        #j = 0
        for patches1, patches2 in pbar:
            #j = j+1
            # ---------------- dataset1 ----------------
            patches1 = patches1.to(device)
            masks1 = create_random_mask(patches1, mask_fraction=mask_fraction).to(device)
            inp1 = patches1.clone()
            inp1 = inp1 * (1.0 - masks1) + 0.5 * masks1
            #print(inp1.shape)
            out1 = model(inp1)
            loss_map1 = mse(out1, patches1).mean(dim=1, keepdim=True)
            pixel_loss1 = (loss_map1 * masks1).sum() / (masks1.sum() + 1e-8)
            fft_score1 = fft_quality(out1, patches1)

            # ---------------- dataset2 ----------------
            patches2 = patches2.to(device)
            masks2 = create_random_mask(patches2, mask_fraction=mask_fraction).to(device)
            inp2 = patches2.clone()
            inp2 = inp2 * (1.0 - masks2) + 0.5 * masks2
            out2 = model(inp2)
            #print(inp1.shape)
            loss_map2 = mse(out2, patches2).mean(dim=1, keepdim=True)
            pixel_loss2 = (loss_map2 * masks2).sum() / (masks2.sum() + 1e-8)
            fft_score2 = fft_quality(out2, patches2)

            # ---------------- combine ----------------
            w_pixel_loss = w1 * pixel_loss1 + w2 * pixel_loss2
            w_fft_score = w1 * fft_score1 + w2 * fft_score2

            if isfft == True:
              loss = w_pixel_loss / (1 + w_fft_score)
            else:
              loss = w_pixel_loss
              w_fft_score = 0
            #loss = w_pixel_loss / (1 + fft_score2)
            optim.zero_grad()
            loss.backward()
            optim.step()

            running_loss += loss.item()
            running_fft_score += w_fft_score
            #running_fft_score += fft_score2

            pbar.set_postfix({
                "loss": f"{running_loss:.4f}",
                "fft_score": f"{running_fft_score / (pbar.n+1):.4f}"
            })
        #print(j)
        scheduler.step()

    return model

# Function to test the GDM model- We call this once we train the GDM and want to denoise a test noisy experimental image. The code returns the denoised image
def denoise_image(model, pil_img, patch_size=128, stride=64):
    model.eval()
    img = pil_img.convert("L")
    arr = np.array(img).astype(np.float32) / 255.0
    H, W = arr.shape
    pad_h = (patch_size - (H % stride)) % stride
    pad_w = (patch_size - (W % stride)) % stride
    padded = np.pad(arr, ((0, pad_h), (0, pad_w)), mode='reflect')
    Ph, Pw = padded.shape
    out_img = np.zeros_like(padded)
    weight = np.zeros_like(padded)
    with torch.no_grad():
        for top in range(0, Ph - patch_size + 1, stride):
            for left in range(0, Pw - patch_size + 1, stride):
                patch = padded[top:top+patch_size, left:left+patch_size]
                x = torch.from_numpy(patch).unsqueeze(0).unsqueeze(0).to(device)
                pred = model(x).cpu().numpy()[0,0]
                out_img[top:top+patch_size, left:left+patch_size] += pred
                weight[top:top+patch_size, left:left+patch_size] += 1.0
    out_img = out_img / (weight + 1e-8)
    out_img = out_img[:H, :W]
    out_img = (out_img * 255.0).clip(0,255).astype(np.uint8)
    return Image.fromarray(out_img)




# Case Study 2
- Pantoea sp. YR343 biofilm images

In [9]:
# 2. Define source folder path (inside your Google Drive)
#    Example: if folder is in "MyDrive/my_project_data"
source_folder = '/content/drive/My Drive/Colab Notebooks/BRAVE Project/AFM/Data/PNG'

# 3. Define destination folder path (inside Colab's working directory)
destination_folder = '/content/AFM_data'

# 4. Copy the folder from Drive to Colab
if os.path.exists(source_folder):
    shutil.copytree(source_folder, destination_folder)
    print(f"Folder copied to {destination_folder}")
else:
    print(f"Source folder not found: {source_folder}")

Folder copied to /content/AFM_data


# Goal-Specific Image Preprocessing
- Disolves the horizontal scan lines (artifacts)
- Preparing the image for channel 1

In [10]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, detrend
from scipy.ndimage import gaussian_filter1d, binary_dilation

def remove_horizontal_lines_darkregions(
    img_path,
    peak_smooth_sigma=3,
    peak_prominence=0.2,
    notch_halfwidth=3,
    dark_percentile=45,
    exclude_center_radius=8,
    show=True
):
    """
    Remove horizontal scan-lines, with special handling to remove them completely
    only in dark regions of the image.

    Parameters
    ----------
    img_path : str
        Path to grayscale AFM/STM image.
    peak_smooth_sigma : float
        Smoothing sigma applied to column-energy before peak detection.
    peak_prominence : float in (0,1)
        Peak detection prominence threshold relative to max column energy.
    notch_halfwidth : int
        Half width (in pixels) of the notch around detected vertical-frequency columns.
    dark_percentile : float (0-100)
        Percentile used to define 'dark' pixels in the image (lower values = fewer dark pixels).
    exclude_center_radius : int
        Number of pixels around the FFT center to exclude from notching (avoids killing DC).
    show : bool
        If True, display diagnostic plots.

    Returns
    -------
    cleaned_uint8 : np.ndarray
        Cleaned AFM/STM image (uint8).
    mask_img : np.ndarray
        FFT mask image (uint8, white=kept, black=masked).
    dark_mask_img : np.ndarray
        Dark-region mask (uint8, white=detrending applied, black=untouched).
    diagnostics : dict
        Useful internals for debugging.
    """

    # --- Load image (grayscale) ---
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE).astype(np.float32)
    H, W = img.shape

    # Normalize to zero-mean
    img_mean, img_std = img.mean(), img.std() + 1e-12
    img_norm = (img - img_mean) / img_std

    # --- FFT ---
    F = np.fft.fftshift(np.fft.fft2(img_norm))
    mag = np.abs(F)
    logmag = np.log1p(mag)

    # Column-wise energy
    col_energy = mag.sum(axis=0)
    col_energy_s = gaussian_filter1d(col_energy, sigma=peak_smooth_sigma)

    # Exclude DC
    cx = W // 2
    exclude_mask_cols = np.zeros_like(col_energy_s, dtype=bool)
    exclude_mask_cols[max(0, cx-exclude_center_radius):min(W, cx+exclude_center_radius+1)] = True
    col_energy_for_peaks = col_energy_s.copy()
    col_energy_for_peaks[exclude_mask_cols] = 0.0

    # Detect streak columns
    height_thresh = peak_prominence * col_energy_for_peaks.max()
    peaks, _ = find_peaks(col_energy_for_peaks, height=height_thresh)
    peaks_all = list(peaks)
    for p in peaks:
        p_sym = 2*cx - p
        if 0 <= p_sym < W:
            peaks_all.append(int(p_sym))
    peaks_all = sorted(set(peaks_all))

    # --- Build FFT mask ---
    mask = np.ones_like(F, dtype=bool)
    for p in peaks_all:
        lo, hi = max(0, p - notch_halfwidth), min(W, p + notch_halfwidth + 1)
        mask[:, lo:hi] = False

    # Keep DC
    cy = H // 2
    Y, X = np.ogrid[:H, :W]
    R = np.sqrt((X-cx)**2 + (Y-cy)**2)
    mask[R < exclude_center_radius] = True

    # Apply mask
    F_notched = F * mask
    img_ifft_notched = np.fft.ifft2(np.fft.ifftshift(F_notched)).real
    img_ifft_rescaled = (img_ifft_notched * img_std) + img_mean

    # --- Dark-region mask ---
    dark_threshold = np.percentile(img_ifft_rescaled, dark_percentile)
    dark_mask = img_ifft_rescaled <= dark_threshold
    dark_mask = binary_dilation(dark_mask, structure=np.ones((3,7)))

    # --- Row-wise detrend in dark regions ---
    img_detrend = np.zeros_like(img_ifft_rescaled)
    for r_idx in range(H):
        row = img_ifft_rescaled[r_idx, :]
        row_detr = detrend(row, type='linear')
        new_row = row.copy()
        new_row[dark_mask[r_idx, :]] = row_detr[dark_mask[r_idx, :]]
        img_detrend[r_idx, :] = new_row

    # Normalize final outputs
    cleaned_uint8 = cv2.normalize(img_detrend, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    mask_img = (mask.astype(np.uint8)) * 255
    dark_mask_img = (dark_mask.astype(np.uint8)) * 255

    diagnostics = {
        "img_norm": img_norm,
        "logmag": logmag,
        "col_energy": col_energy,
        "col_energy_s": col_energy_s,
        "detected_peaks": peaks_all,
        "fft_mask": mask,
        "dark_mask": dark_mask
    }

    if show:
        plt.figure(figsize=(18,6))
        plt.subplot(1,4,1); plt.imshow(img, cmap='gray'); plt.title("Original"); plt.axis("off")
        plt.subplot(1,4,2); plt.imshow(cleaned_uint8, cmap='gray'); plt.title("Cleaned"); plt.axis("off")
        plt.subplot(1,4,3); plt.imshow(mask_img, cmap='gray'); plt.title("FFT Mask"); plt.axis("off")
        plt.subplot(1,4,4); plt.imshow(dark_mask_img, cmap='gray'); plt.title("Dark Mask"); plt.axis("off")
        plt.show()

    return cleaned_uint8, mask_img, dark_mask_img, diagnostics


In [ ]:
train_image_path = "/content/AFM_data/afm_image1.png"
#Example usage (adjust path and parameters to taste):
cleaned, masked, dark_mask, diag = remove_horizontal_lines_darkregions(train_image_path,
                                                    peak_smooth_sigma=0.1,
                                                    peak_prominence=0.2,
                                                    notch_halfwidth=1,
                                                    dark_percentile=50,
                                                    exclude_center_radius=50,
                                                    show=True)

In [12]:
cleaned_image_pil = Image.fromarray(cleaned)
dark_mask_pil = Image.fromarray(dark_mask)

# Save as .png files
cleaned_image_path = "cleaned_image.png"
dark_mask_path = "dark_mask.png"

cleaned_image_pil.save(cleaned_image_path)
dark_mask_pil.save(dark_mask_path)

# Goal-Specific Image Preprocessing
- Merging the original image with dark mask to preserve the features in non-black region
- Preparing the image for channel 2

#

In [13]:
import numpy as np
from PIL import Image

def merge_with_mask(original_path, mask_path, output_path="merged.png"):
    """
    Merge two images:
      - Dark regions (black in mask) → replaced by white
      - Non-dark regions (white in mask) → original image is kept
    """
    # Load images
    original = Image.open(original_path).convert("L")  # grayscale
    mask = Image.open(mask_path).convert("L")          # grayscale mask

    # Convert to numpy
    orig_arr = np.array(original)
    mask_arr = np.array(mask)

    # Ensure same shape
    min_h = min(orig_arr.shape[0], mask_arr.shape[0])
    min_w = min(orig_arr.shape[1], mask_arr.shape[1])
    orig_arr = orig_arr[:min_h, :min_w]
    mask_arr = mask_arr[:min_h, :min_w]

    # White areas in mask → keep original
    # Black areas in mask → replace with white
    merged_arr = np.where(mask_arr > 128, orig_arr, 255)

    # Convert back to image
    merged_img = Image.fromarray(merged_arr.astype(np.uint8))
    merged_img.save(output_path)

    return merged_img

# Example usage (with your split images)
# merged = merge_with_mask("original.png", "dark_mask.png", "merged.png")
# merged.show()


In [14]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def merge_shadow_with_mask(original_path, mask_path, output_path="merged.png"):
    """
    Replaces the shadowed (scan line) regions of the original image
    with white where the mask has white.
    """
    # Load original and mask
    original = Image.open(original_path).convert("L")  # grayscale
    mask = Image.open(mask_path).convert("L")          # grayscale mask

    # Convert to numpy
    orig_arr = np.array(original)
    mask_arr = np.array(mask)

    # Ensure same size
    min_h = min(orig_arr.shape[0], mask_arr.shape[0])
    min_w = min(orig_arr.shape[1], mask_arr.shape[1])
    orig_arr = orig_arr[:min_h, :min_w]
    mask_arr = mask_arr[:min_h, :min_w]

    # White in mask → replace with white
    # Black in mask → keep original
    merged_arr = np.where(mask_arr > 128, 255, orig_arr)

    # Save merged result
    merged_img = Image.fromarray(merged_arr.astype(np.uint8))
    merged_img.save(output_path)

    # Optional visualization
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1); plt.imshow(orig_arr, cmap="gray"); plt.title("Original"); plt.axis("off")
    plt.subplot(1,3,2); plt.imshow(mask_arr, cmap="gray"); plt.title("Dark Mask"); plt.axis("off")
    plt.subplot(1,3,3); plt.imshow(merged_arr, cmap="gray"); plt.title("Merged Result"); plt.axis("off")
    plt.tight_layout()
    plt.show()

    return merged_img

# Example usage:
# merge_shadow_with_mask("original.png", "dark_mask.png", "merged.png")


# Final preprocessed images from single experimental image

In [ ]:
train_image_path1 = "/content/AFM_data/afm_image1.png"
train_image_path2 = "/content/dark_mask.png"

merged = merge_shadow_with_mask(train_image_path1, train_image_path2, "merged.png")
merged.show()

Training model - w1=0.5, w2=0.5

In [ ]:
# -------------------- Hyperparameters --------------------
#train_image_path = "../Desktop/Dataset1_TaS2/simulate_si.png"  # single training image
#train_image_path1 = "/content/AFM_data/afm_image6.png"
#train_image_path2 = "/content/AFM_data/afm_image1.png"
train_image_path1 = "/content/merged.png"
train_image_path2 = "/content/dark_mask.png"
patch_size = 128
batch_size = 8
epochs = 50
lr = 1e-3
mask_fraction = 0.1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# w1 = 0.9
# w2 = 1 - w1
# ---- Run ----
model2 = train(w1=0.5, w2=0.5)

Testing model -Denoise experimental images

In [ ]:
# Base path & filename parts
#base_dir = "../Desktop/Dataset1_TaS2/si_expe"
base_dir = "/content/AFM_data/"
prefix = "afm_image"
suffix = ".png"

# Loop through sequential file numbers
for i in range(1, 17):  # adjust 5 to your total + 1
    file_name = f"{prefix}{i:d}{suffix}"  # 1, 2, 3...
    image_path = os.path.join(base_dir, file_name)
    if os.path.exists(image_path):
            img = Image.open(image_path).convert("RGB")
            width, height = img.size
            den = denoise_image(model2, img, patch_size=width-1, stride=patch_size//2)
            showFFTScore(img,den)
            show_detected_lines_angle_range(img.convert("L"),  sigma=1, low_threshold=1, high_threshold=10, line_length=1, line_gap=2, angle_min=-10,  angle_max=10)
            show_detected_lines_angle_range(den.convert("L"),  sigma=1, low_threshold=1, high_threshold=10, line_length=1, line_gap=2, angle_min=-10,  angle_max=10)

Training model - w1=0, w2=1

In [ ]:
# -------------------- Hyperparameters --------------------
#train_image_path = "../Desktop/Dataset1_TaS2/simulate_si.png"  # single training image
#train_image_path1 = "/content/AFM_data/afm_image6.png"
#train_image_path2 = "/content/AFM_data/afm_image1.png"
train_image_path1 = "/content/merged.png"
train_image_path2 = "/content/dark_mask.png"
patch_size = 128
batch_size = 8
epochs = 50
lr = 1e-3
mask_fraction = 0.1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# w1 = 0.9
# w2 = 1 - w1
# ---- Run ----
model2 = train(w1=0.00, w2=1.0)
#

Testing model -Denoise all experimental images

In [ ]:
# Base path & filename parts
#base_dir = "../Desktop/Dataset1_TaS2/si_expe"
base_dir = "/content/AFM_data/"
prefix = "afm_image"
suffix = ".png"

# Loop through sequential file numbers
for i in range(1, 17):  # adjust 5 to your total + 1
    file_name = f"{prefix}{i:d}{suffix}"  # 1, 2, 3...
    image_path = os.path.join(base_dir, file_name)
    if os.path.exists(image_path):
            img = Image.open(image_path).convert("RGB")
            width, height = img.size
            den = denoise_image(model2, img, patch_size=width-1, stride=patch_size//2)
            showFFTScore(img,den)
            show_detected_lines_angle_range(img.convert("L"),  sigma=1, low_threshold=5, high_threshold=10, line_length=1, line_gap=1, angle_min=-10,  angle_max=10)
            show_detected_lines_angle_range(den.convert("L"),  sigma=1, low_threshold=5, high_threshold=10, line_length=1, line_gap=1, angle_min=-10,  angle_max=10)

Training model - w1=0.1, w2=0.9

In [ ]:
# -------------------- Hyperparameters --------------------
#train_image_path = "../Desktop/Dataset1_TaS2/simulate_si.png"  # single training image
#train_image_path1 = "/content/AFM_data/afm_image6.png"
#train_image_path2 = "/content/AFM_data/afm_image1.png"
train_image_path1 = "/content/merged.png"
train_image_path2 = "/content/dark_mask.png"
patch_size = 128
batch_size = 8
epochs = 50
lr = 1e-3
mask_fraction = 0.1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# w1 = 0.9
# w2 = 1 - w1
# ---- Run ----
model2 = train(w1=0.1, w2=0.9)

Testing model -Denoise all experimental images

In [ ]:
# Base path & filename parts
#base_dir = "../Desktop/Dataset1_TaS2/si_expe"
base_dir = "/content/AFM_data/"
prefix = "afm_image"
suffix = ".png"

# Loop through sequential file numbers
for i in range(1, 17):  # adjust 5 to your total + 1
    file_name = f"{prefix}{i:d}{suffix}"  # 1, 2, 3...
    image_path = os.path.join(base_dir, file_name)
    if os.path.exists(image_path):
            img = Image.open(image_path).convert("RGB")
            width, height = img.size
            den = denoise_image(model2, img, patch_size=width-1, stride=patch_size//2)
            showFFTScore(img,den)
            show_detected_lines_angle_range(img.convert("L"),  sigma=1, low_threshold=5, high_threshold=10, line_length=1, line_gap=1, angle_min=-10,  angle_max=10)
            show_detected_lines_angle_range(den.convert("L"),  sigma=1, low_threshold=5, high_threshold=10, line_length=1, line_gap=1, angle_min=-10,  angle_max=10)

Training model - w1=0.01, w2=0.99

In [ ]:
# -------------------- Hyperparameters --------------------
#train_image_path = "../Desktop/Dataset1_TaS2/simulate_si.png"  # single training image
#train_image_path1 = "/content/AFM_data/afm_image6.png"
#train_image_path2 = "/content/AFM_data/afm_image1.png"
train_image_path1 = "/content/merged.png"
train_image_path2 = "/content/dark_mask.png"
patch_size = 128
batch_size = 8
epochs = 50
lr = 1e-3
mask_fraction = 0.1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# w1 = 0.9
# w2 = 1 - w1
# ---- Run ----
model2 = train(w1=0.01, w2=0.99)

Testing model- Denoise all experimental images

In [ ]:
# Base path & filename parts
#base_dir = "../Desktop/Dataset1_TaS2/si_expe"
base_dir = "/content/AFM_data/"
prefix = "afm_image"
suffix = ".png"

# Loop through sequential file numbers
for i in range(1, 17):  # adjust 5 to your total + 1
    file_name = f"{prefix}{i:d}{suffix}"  # 1, 2, 3...
    image_path = os.path.join(base_dir, file_name)
    if os.path.exists(image_path):
            img = Image.open(image_path).convert("RGB")
            width, height = img.size
            den = denoise_image(model2, img, patch_size=width-1, stride=patch_size//2)
            showFFTScore(img,den)
            show_detected_lines_angle_range(img.convert("L"),  sigma=1, low_threshold=5, high_threshold=10, line_length=1, line_gap=1, angle_min=-10,  angle_max=10)
            show_detected_lines_angle_range(den.convert("L"),  sigma=1, low_threshold=5, high_threshold=10, line_length=1, line_gap=1, angle_min=-10,  angle_max=10)

Training model - w1=0.9, w2=0.1

In [ ]:
# -------------------- Hyperparameters --------------------
#train_image_path = "../Desktop/Dataset1_TaS2/simulate_si.png"  # single training image
#train_image_path1 = "/content/AFM_data/afm_image6.png"
#train_image_path2 = "/content/AFM_data/afm_image1.png"
train_image_path1 = "/content/merged.png"
train_image_path2 = "/content/dark_mask.png"
patch_size = 128
batch_size = 8
epochs = 50
lr = 1e-3
mask_fraction = 0.1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# w1 = 0.9
# w2 = 1 - w1
# ---- Run ----
model2 = train(w1=0.9, w2=0.1)

Testing model - Denoise all experimental images

In [ ]:
# Base path & filename parts
#base_dir = "../Desktop/Dataset1_TaS2/si_expe"
base_dir = "/content/AFM_data/"
prefix = "afm_image"
suffix = ".png"

# Loop through sequential file numbers
for i in range(1, 17):  # adjust 5 to your total + 1
    file_name = f"{prefix}{i:d}{suffix}"  # 1, 2, 3...
    image_path = os.path.join(base_dir, file_name)
    if os.path.exists(image_path):
            img = Image.open(image_path).convert("RGB")
            width, height = img.size
            den = denoise_image(model2, img, patch_size=width-1, stride=patch_size//2)
            showFFTScore(img,den)
            show_detected_lines_angle_range(img.convert("L"),  sigma=1, low_threshold=5, high_threshold=10, line_length=1, line_gap=1, angle_min=-10,  angle_max=10)
            show_detected_lines_angle_range(den.convert("L"),  sigma=1, low_threshold=5, high_threshold=10, line_length=1, line_gap=1, angle_min=-10,  angle_max=10)

Training model - w1=1.0, w2=0.0

In [ ]:
# -------------------- Hyperparameters --------------------
#train_image_path = "../Desktop/Dataset1_TaS2/simulate_si.png"  # single training image
#train_image_path1 = "/content/AFM_data/afm_image6.png"
#train_image_path2 = "/content/AFM_data/afm_image1.png"
train_image_path1 = "/content/merged.png"
train_image_path2 = "/content/dark_mask.png"
patch_size = 128
batch_size = 8
epochs = 50
lr = 1e-3
mask_fraction = 0.1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# w1 = 0.9
# w2 = 1 - w1
# ---- Run ----
model2 = train(w1=1.0, w2=0.0)

Testing model - Denoise all experimental images

In [ ]:
# Base path & filename parts
#base_dir = "../Desktop/Dataset1_TaS2/si_expe"
base_dir = "/content/AFM_data/"
prefix = "afm_image"
suffix = ".png"

# Loop through sequential file numbers
for i in range(1, 17):  # adjust 5 to your total + 1
    file_name = f"{prefix}{i:d}{suffix}"  # 1, 2, 3...
    image_path = os.path.join(base_dir, file_name)
    if os.path.exists(image_path):
            img = Image.open(image_path).convert("RGB")
            width, height = img.size
            den = denoise_image(model2, img, patch_size=width-1, stride=patch_size//2)
            showFFTScore(img,den)
            show_detected_lines_angle_range(img.convert("L"),  sigma=1, low_threshold=5, high_threshold=10, line_length=1, line_gap=1, angle_min=-10,  angle_max=10)
            show_detected_lines_angle_range(den.convert("L"),  sigma=1, low_threshold=5, high_threshold=10, line_length=1, line_gap=1, angle_min=-10,  angle_max=10)

Training model - w1=0.5, w2=0.5 without FFT loss

In [ ]:
# -------------------- Hyperparameters --------------------
#train_image_path = "../Desktop/Dataset1_TaS2/simulate_si.png"  # single training image
#train_image_path1 = "/content/AFM_data/afm_image6.png"
#train_image_path2 = "/content/AFM_data/afm_image1.png"
train_image_path1 = "/content/merged.png"
train_image_path2 = "/content/dark_mask.png"
patch_size = 128
batch_size = 8
epochs = 50
lr = 1e-3
mask_fraction = 0.1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# w1 = 0.9
# w2 = 1 - w1
# ---- Run ----
model2 = train(w1=0.5, w2=0.5, isfft=False)

Testing model -Denoise all experimental images

In [ ]:
# Base path & filename parts
#base_dir = "../Desktop/Dataset1_TaS2/si_expe"
base_dir = "/content/AFM_data/"
prefix = "afm_image"
suffix = ".png"

# Loop through sequential file numbers
for i in range(1, 17):  # adjust 5 to your total + 1
    file_name = f"{prefix}{i:d}{suffix}"  # 1, 2, 3...
    image_path = os.path.join(base_dir, file_name)
    if os.path.exists(image_path):
            img = Image.open(image_path).convert("RGB")
            width, height = img.size
            den = denoise_image(model2, img, patch_size=width-1, stride=patch_size//2)
            showFFTScore(img,den)
            show_detected_lines_angle_range(img.convert("L"),  sigma=1, low_threshold=5, high_threshold=10, line_length=1, line_gap=1, angle_min=-10,  angle_max=10)
            show_detected_lines_angle_range(den.convert("L"),  sigma=1, low_threshold=5, high_threshold=10, line_length=1, line_gap=1, angle_min=-10,  angle_max=10)